# RECONOCIMIENTO DE EXPRESIONES FACIALES 

Librerias 

In [76]:
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import random

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix

In [77]:
# Cargar los archivos CSV
train_df = pd.read_csv('./data/train_set.csv')
test_df = pd.read_csv('./data/test_set.csv')

In [78]:
# Función para cargar y preprocesar imágenes
def load_and_preprocess_image(image_path, target_size=(48, 48)):
    try:
        image = load_img(image_path, color_mode='grayscale', target_size=target_size)
        image = img_to_array(image)
        image = image / 255.0  # Normalizar los valores de los píxeles
        return image
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

In [79]:
# Función para cargar imágenes y etiquetas desde una carpeta
def load_images_from_folder(folder_path):
    images = []
    labels = []
    label_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}  # Asignar etiquetas numéricas a las emociones
    for emotion in os.listdir(folder_path):
        emotion_folder = os.path.join(folder_path, emotion)
        if os.path.isdir(emotion_folder):
            for img_name in os.listdir(emotion_folder):
                img_path = os.path.join(emotion_folder, img_name)
                if os.access(img_path, os.R_OK):  # Verificar permisos de lectura
                    image = load_and_preprocess_image(img_path)
                    if image is not None:
                        images.append(image)
                        labels.append(label_map[emotion])
                else:
                    print(f"Permission denied for image {img_path}")
    return np.array(images), np.array(labels)

In [80]:
# Cargar las imágenes de entrenamiento y las etiquetas
train_images, train_labels = load_images_from_folder('./data/images/train')

In [81]:
# Convertir las etiquetas a formato categórico
train_labels = to_categorical(train_labels, num_classes=7)

In [82]:
# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [83]:
print(f"Train images shape: {X_train.shape}, Train labels shape: {y_train.shape}")
print(f"Validation images shape: {X_val.shape}, Validation labels shape: {y_val.shape}")

Train images shape: (23056, 48, 48, 1), Train labels shape: (23056, 7)
Validation images shape: (5765, 48, 48, 1), Validation labels shape: (5765, 7)


In [84]:
# Cargar las imágenes de prueba
def load_test_images(folder_path):
    images = []
    image_names = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        if os.access(img_path, os.R_OK):  # Verificar permisos de lectura
            image = load_and_preprocess_image(img_path)
            if image is not None:
                images.append(image)
                image_names.append(img_name)
        else:
            print(f"Permission denied for image {img_path}")
    return np.array(images), image_names



In [85]:
test_images, test_image_names = load_test_images('./data/images/test')
print(f"Test images shape: {test_images.shape}")

Error loading image ./data/images/test\temp: [Errno 13] Permission denied: './data/images/test\\temp'
Test images shape: (7066, 48, 48, 1)


In [86]:
def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')  # Usamos softmax para la clasificación de 7 clases
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [87]:
# Definir el tamaño de entrada basado en nuestras imágenes
input_shape = (48, 48, 1)
model = create_cnn_model(input_shape)
model.summary()

c:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 355,847 (1.36 MB)

 Trainable params: 355,847 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

In [88]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.2309 - loss: 1.8298 - val_accuracy: 0.3670 - val_loss: 1.6448
Epoch 2/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.3541 - loss: 1.6301 - val_accuracy: 0.4409 - val_loss: 1.4580
Epoch 3/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.4167 - loss: 1.5111 - val_accuracy: 0.4751 - val_loss: 1.3697
Epoch 4/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.4415 - loss: 1.4354 - val_accuracy: 0.4961 - val_loss: 1.3331
Epoch 5/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.4646 - loss: 1.3860 - val_accuracy: 0.5112 - val_loss: 1.2806
Epoch 6/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.4910 - loss: 1.3376 - val_accuracy: 0.5239 - val_loss: 1.2574
Epoch 7/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.5029 - loss: 1.3143 - val_accuracy: 0.5278 - val_loss: 1.2449
Epoch 8/25
721/721 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.5107 - loss: 1.2809 - 

In [89]:
# Evaluar el modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

181/181 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5704 - loss: 1.1297
Validation Loss: 1.1447548866271973, Validation Accuracy: 0.5646140575408936


In [90]:
# Realizar predicciones en el conjunto de pruebas
test_predictions = model.predict(test_images)
test_predictions = np.argmax(test_predictions, axis=1)  # Convertir a etiquetas

221/221 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [91]:
# Crear un diccionario inverso para convertir etiquetas numéricas a etiquetas de emociones
inv_label_map = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

In [92]:
# Convertir etiquetas numéricas a etiquetas de emociones
test_predictions_labels = [inv_label_map[pred] for pred in test_predictions]

In [93]:
sample_submission_df = pd.read_csv('./data/sample_submision.csv')

In [94]:
# Crear un diccionario con las predicciones
predictions_dict = dict(zip(test_image_names, test_predictions_labels))

In [100]:
# Asignar las predicciones en el orden correcto según sample_submission.csv
sample_submission_df['emotion'] = sample_submission_df['id_img'].map(predictions_dict)


In [99]:
# Save the submission file with the ordered predictions
submission_file_path = './data/submission.csv'
sample_submission_df.to_csv(submission_file_path, index=False)
print(f"Submission file saved to {submission_file_path}")
print(sample_submission_df.head())


Submission file saved to ./data/submission.csv
   id_img     label emotion
0   10052  surprise     NaN
1   10065  surprise     NaN
2   10079  surprise     NaN
3   10095  surprise     NaN
4   10121  surprise     NaN
